In [1]:
%load_ext autoreload
%autoreload 2

import uproot as ur
import awkward as ak
import numpy as np

mac = False

data_path = '/fast_scratch/atlas_images/v01-45/'
if mac:
    data_path = '/Users/swiatlow/Data/caloml/graph_data/'

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "7"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import tensorflow as tf

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [4]:
X = np.load(data_path+'X.npy')

In [5]:
Y = tf.keras.utils.to_categorical(np.concatenate((np.ones(90000), np.zeros(90000))))

In [6]:
Phi_sizes, F_sizes = (50, 50, 64), (50, 50, 50)


In [7]:
from energyflow.utils import data_split

# do train/val/test split 
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(X, Y, val=10000, test=40000)

print('Done train/val/test split')

Done train/val/test split


In [8]:
pfn = PFN(input_dim=X.shape[-1], Phi_sizes=Phi_sizes, F_sizes=F_sizes)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
tdist_0 (TimeDistributed)       (None, None, 50)     200         input[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, None, 50)     0           tdist_0[0][0]                    
__________________________________________________________________________________________________
tdist_1 (TimeDistributed)       (None, None, 50)     2550        activation[0][0]                 
______________________________________________________________________________________________

In [9]:
# train model
history = pfn.fit(X_train, Y_train,
        epochs=100,
        batch_size=500,
        validation_data=(X_val, Y_val),
        verbose=1)

Epoch 1/50
260/260 [==============================] - 10s 30ms/step - loss: 6.3019 - acc: 0.5695 - val_loss: 0.5772 - val_acc: 0.7296
Epoch 2/50
260/260 [==============================] - 7s 28ms/step - loss: 0.7875 - acc: 0.7328 - val_loss: 0.6091 - val_acc: 0.7945
Epoch 3/50
260/260 [==============================] - 8s 32ms/step - loss: 0.6436 - acc: 0.7598 - val_loss: 0.7578 - val_acc: 0.6664
Epoch 4/50
260/260 [==============================] - 7s 29ms/step - loss: 0.7102 - acc: 0.7359 - val_loss: 0.4961 - val_acc: 0.8017
Epoch 5/50
260/260 [==============================] - 7s 29ms/step - loss: 0.6080 - acc: 0.7564 - val_loss: 0.5906 - val_acc: 0.7820
Epoch 6/50
260/260 [==============================] - 7s 28ms/step - loss: 0.5482 - acc: 0.7590 - val_loss: 0.5911 - val_acc: 0.7798
Epoch 7/50
260/260 [==============================] - 7s 28ms/step - loss: 0.5074 - acc: 0.7713 - val_loss: 0.4437 - val_acc: 0.7953
Epoch 8/50
260/260 [==============================] - 7s 28ms/step -

In [10]:
preds = pfn.predict(X_test, batch_size=1000)

In [11]:
pfn_fp, pfn_tp, threshs = roc_curve(Y_test[:,1], preds[:,1])

In [12]:
# get area under the ROC curve
auc = roc_auc_score(Y_test[:,1], preds[:,1])
print()
print('PFN AUC:', auc)
print()


PFN AUC: 0.9548550671434395



In [ ]:
    plt.cla(); plt.clf()
    fig = plt.figure()
    fig.patch.set_facecolor('white')

    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig('Plots/accuracy_' + layer + '.pdf')
    plt.show()


    # summarize history for loss
    fig = plt.figure()
    fig.patch.set_facecolor('white')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig(plotpath + 'loss_' + layer + '.pdf')
    plt.show()